# Get Geodata for Properties

This routine will call the google geocode API to get the latitude and lomgitude based on address passed.


In [2]:
import pandas as pd
import numpy as np

import requests
import googlemaps
import time

In [31]:
apikey = 'removed key'


In [32]:
def combine_address(row):
    sub_address = row['sub_address']
    city = row['City Name']
    state = 'CO'
 
    sub_address = sub_address.replace(" ", "+")
    city = city.replace(" ", "+")

    return sub_address+"+"+city+"+"+state
    

In [33]:
def create_sub_address(x):
    
    x_list = x.split()

    sub_address = ""
    for ndx in x_list:
        if ndx == 'Unit' or ndx == "#":
            break
        else:
            sub_address = sub_address + " " + ndx

    return sub_address[1:]

In [34]:
def get_lat (geo):
    return geo['lat']

def get_lng (geo):
    return geo['lng']

In [35]:
def get_geo_code(address_search, user_apikey):
    
    # Instantiate google maps 
    gmaps = googlemaps.Client(key=user_apikey)

    try:
        results = gmaps.geocode(address=address_search)
    
        # to prevent overloading the call to google places_nearby
        time.sleep(2)

        lat = results[0]["geometry"]["location"]["lat"]
        lng = results[0]["geometry"]["location"]["lng"] 
    
    except:
        
        lat = 10
        lng = 10
    
        
    return {'lat': lat, 'lng':lng}


In [36]:
# Read in data from .
df_data = pd.read_csv('../rawdata/Summit_County_public_record_2017_2019.csv')

df_data.head()

,Tax ID,Address,City Name,Property Zip,Beds,Baths (Total),Building SqFt - Total,Yr Built,Land Use - CoreLogic,Sale Date,Last Sale Price,File
0,6508905,910 Copper RD Unit #210,Frisco,80443,1,2,790,2000,High Rise Condo,2/10/17,420000,2017
1,400372,317 S Fuller Placer RD,Breckenridge,80424,3,5,4557,2018,Sfr,10/18/17,275100,2017
2,3001037,060 Tennis Club RD Unit #1605,Dillon,80435,1,1,918,1977,Condominium,7/21/17,264000,2017
3,400233,1173 Ski Hill RD Unit #126,Breckenridge,80424,2,2,780,1972,Condominium,5/4/17,380000,2017
4,3001255,22320 Us Highway 6 Unit #1766,Dillon,80435,1,2,1168,1977,Condominium,7/13/17,395000,2017


In [37]:
df_data.isnull().sum()

Tax ID                   0
Address                  0
City Name                0
Property Zip             0
Beds                     0
Baths (Total)            0
Building SqFt - Total    0
Yr Built                 0
Land Use - CoreLogic     0
Sale Date                0
Last Sale Price          0
File                     0
dtype: int64

In [38]:
df_data['sub_address'] = df_data['Address'].apply(lambda x: create_sub_address(x))
df_data['search_by_address'] = df_data.apply(lambda row: combine_address (row), axis = 1)

In [39]:
df_data.isnull().sum()

Tax ID                   0
Address                  0
City Name                0
Property Zip             0
Beds                     0
Baths (Total)            0
Building SqFt - Total    0
Yr Built                 0
Land Use - CoreLogic     0
Sale Date                0
Last Sale Price          0
File                     0
sub_address              0
search_by_address        0
dtype: int64

In [40]:
df_data.shape

(5490, 14)

In [41]:
# reset index
# df_data.sort_values('sub_address', inplace = True)
# df_data.drop_duplicates(subset = 'sub_address', keep = 'first', inplace = True) 
df_data.reset_index(drop = True, inplace = True)

In [48]:
df_data.shape

(5490, 17)

In [43]:
print(f'Here we go...')
df_data['geometry'] = df_data['search_by_address'].apply(lambda x: get_geo_code(x, apikey))
print(f'done!')

Here we go...
done!


In [44]:
df_data['lat'] = df_data['geometry'].apply(lambda x: get_lat(x))
df_data['lng'] = df_data['geometry'].apply(lambda x: get_lng(x))

In [45]:
df_data.head()

,Tax ID,Address,City Name,Property Zip,Beds,Baths (Total),Building SqFt - Total,Yr Built,Land Use - CoreLogic,Sale Date,Last Sale Price,File,sub_address,search_by_address,geometry,lat,lng
0,6508905,910 Copper RD Unit #210,Frisco,80443,1,2,790,2000,High Rise Condo,2/10/17,420000,2017,910 Copper RD,910+Copper+RD+Frisco+CO,"{'lat': 39.501766, 'lng': -106.157572}",39.501766,-106.157572
1,400372,317 S Fuller Placer RD,Breckenridge,80424,3,5,4557,2018,Sfr,10/18/17,275100,2017,317 S Fuller Placer RD,317+S+Fuller+Placer+RD+Breckenridge+CO,"{'lat': 39.4660605, 'lng': -106.0110134}",39.466060,-106.011013
2,3001037,060 Tennis Club RD Unit #1605,Dillon,80435,1,1,918,1977,Condominium,7/21/17,264000,2017,060 Tennis Club RD,060+Tennis+Club+RD+Dillon+CO,"{'lat': 39.6087488, 'lng': -105.9748657}",39.608749,-105.974866
3,400233,1173 Ski Hill RD Unit #126,Breckenridge,80424,2,2,780,1972,Condominium,5/4/17,380000,2017,1173 Ski Hill RD,1173+Ski+Hill+RD+Breckenridge+CO,"{'lat': 39.481508, 'lng': -106.059609}",39.481508,-106.059609
4,3001255,22320 Us Highway 6 Unit #1766,Dillon,80435,1,2,1168,1977,Condominium,7/13/17,395000,2017,22320 Us Highway 6,22320+Us+Highway+6+Dillon+CO,"{'lat': 39.6068755, 'lng': -105.9649734}",39.606876,-105.964973


In [46]:
df_data.isnull().sum()

Tax ID                   0
Address                  0
City Name                0
Property Zip             0
Beds                     0
Baths (Total)            0
Building SqFt - Total    0
Yr Built                 0
Land Use - CoreLogic     0
Sale Date                0
Last Sale Price          0
File                     0
sub_address              0
search_by_address        0
geometry                 0
lat                      0
lng                      0
dtype: int64

In [47]:
df_data['lat'].value_counts()

39.481654    71
39.607539    40
39.607066    35
39.501766    35
39.474061    34
             ..
39.501570     1
39.617576     1
39.382949     1
39.486767     1
39.629056     1
Name: lat, Length: 3081, dtype: int64

In [49]:
# save data frame, cleaned.
df_data.to_csv('../rawdata/Summit_County_public_record_with_geo_2017_2019.csv', index = False)
